In [35]:
import tensorflow as tf

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
print(tf.__version__)

In [40]:
pwd

'/home/mnolte/repos/river_forecast/jupyter-notebooks'

In [41]:
model = tf.keras.models.load_model('../models/LSTM_model_v2')

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              multiple                  0         
_________________________________________________________________
conv1d (Conv1D)              multiple                  256       
_________________________________________________________________
lstm (LSTM)                  multiple                  328704    
_________________________________________________________________
lstm_1 (LSTM)                multiple                  197120    
_________________________________________________________________
repeat_vector (RepeatVector) multiple                  0         
_________________________________________________________________
lstm_2 (LSTM)                multiple                  131584    
_________________________________________________________________
lstm_3 (LSTM)                multiple                  4

In [43]:
from river_forecast.api_data_access import RivermapDataRetriever

In [44]:
rivermap_data = RivermapDataRetriever()

In [45]:
flow_df = rivermap_data.get_latest_river_flow(n_days=4, station='Dranse')

In [20]:
def multi_step_forecast(time_series, model, n_steps=12):
    time_series = np.array(time_series).reshape(1, time_series.size, 1)
    forecast = np.zeros(n_steps, dtype=float)
    for i in range(n_steps):
        time_series = model.predict(time_series)
        forecast[i] = time_series[0, -1, 0]
    return forecast

In [ ]:
def multi_step_forecast_from_diff(time_series, model, last_value, n_steps=12):
    forecast = multi_step_forecast(time_series, model, n_steps=n_steps)
    f_t = last_value
    for i in range(n_steps):
        forecast[i] = f_t + forecast[i]
        f_t = forecast[i]
    return forecast

In [46]:
def multi_step_forecast_auto_encoder(time_series, model):
    time_series = np.array(time_series).reshape(1, time_series.size, 1)
    forecast = model.predict(time_series)
    return forecast

In [28]:
multi_step_forecast_from_diff(flow_df.diff(periods=1).dropna(), model, flow_df.iloc[-1]['discharge'], n_steps=12)

array([12.13745576, 12.07853703, 12.22676003, 12.5814679 , 13.15584079,
       13.92868921, 14.83726123, 15.75773898, 16.52067694, 16.9779374 ,
       17.10436967, 16.89248848])

In [47]:
multi_step_forecast_auto_encoder(flow_df.iloc[-24:], model)

array([[[ 5.175497],
        [ 5.852569],
        [ 8.549745],
        [10.873962],
        [11.471116],
        [11.40634 ]]], dtype=float32)